## Splitting all available data
#### 133 subjects in this dataset

In [ ]:
import glob, os
import numpy as np
import pandas as pd
import nibabel as nib

In [ ]:
#FILENAME="all_batch1.nii.gz"

DERIV_DIR = '/projects/niblab/bids_projects/Experiments/ChocoData/derivatives'
outpath = "/projects/niblab/bids_projects/Experiments/ChocoData/images"

sub_list=[]
all_func = sorted(glob.glob(os.path.join(DERIV_DIR, "sub-*", "ses-*", "func/Analysis/feat1/task*/filtered_func_data.nii.gz")))
for x in all_func:
    if x.split("/")[-7] not in sub_list:
        sub_list.append(x.split("/")[-7])
        #print(sub_list)

    
chunks = [sub_list[x:x+7] for x in range(0, len(sub_list), 7)]
for i,chunk in enumerate(chunks):
    batch = i + 1
    filename = "all_waves_b%s"%(batch)
    nifti_name = filename+".nii.gza"
    print(filename)
    #print(nifti_name)
    if os.path.exists(os.path.join(outpath,nifti_name)):
        pass
    else:
        print('here')
        all_func = []
        subj_dict = {}
        for x in chunk: 
            subj_dict[x] = [] 
        #print(subj_dict)
    
    
# iterate through subjects in chunk list
        for sub_id in chunk:
            all_funcs = sorted(glob.glob(os.path.join(DERIV_DIR, sub_id, "ses-*", "func/Analysis/feat1/task*/filtered_func_data.nii.gz")))
# add each file from the functionals to a temp list to make nifti image
            for file in all_funcs:
                #print(file)
                all_func.append(file)
        #count=0
        #for fun_ in all_func:
         #   print(fun_)
          #  count = count +1 
        #print(count)


#all_func = sorted(all_func[:54])

#load in all the files from the glob above, then convert them from nifti1 to nifti2\n",
        ni2_funcs = (nib.Nifti2Image.from_image(nib.load(func)) for func in all_func)
#concat, this is with nibabel, but should work with nilearn too
        ni2_concat = nib.concat_images(ni2_funcs, check_affines=False, axis=3)
#set the output file name

        outfile=os.path.join(outpath,nifti_name)
#write the file
        ni2_concat.to_filename(outfile)
    
    
    # -add corresponding milkshake label    
        ct=0
        for d in subj_dict:
            task_dirs = glob.glob(os.path.join('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives', d, "ses-*/func/Analysis/feat1/*"))
            for task in task_dirs:
                dir_name = task.split("/")[-1]
                sess = task.split("/")[-5]
                mlk = dir_name.split(".")[0].split("e")[1]
                id_ = sess+"_"+mlk
                subj_dict[d].append(id_)
                ct=ct+1
        
        temp_out = "/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/temp"
        for sub in subj_dict:
            file = open(temp_out+"/tempfile_%s.txt"%sub, 'a')
            for label in sorted(subj_dict[sub]):
                fileX = '/projects/niblab/data/eric_data/ev_files/milkshake/mk%s_attr.txt'%label.split("_")[1]
                fileX_contents = open(fileX, 'r')
                data = fileX_contents.read()
                fileX_contents.close()
                file.write(data)
            file.close()
    
        tempfiles=sorted(glob.glob("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/temp/tempfile_*.txt"))

# fill in 20% of test files 
        test_slice_count = round(len(tempfiles) *.2)
        test_slice = chunk[:test_slice_count]
        for sub in test_slice:
            df=pd.read_csv("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/temp/tempfile_%s.txt"%sub, sep=' ',header=None)
            df[1].replace(0,1, inplace=True)
            print(df.head())
            df.to_csv("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/temp/tempfile_%s.txt"%sub, sep=' ', index=False, header=None)    

# Make initial text file    
        fileout = open("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/%s.txt"%(filename), "a")
        fileout.write("Label sess\n")
       
        for f in tempfiles:
            print("adding file %s"%f)
            f_contents = open(f, "r")
            data=f_contents.read()
            f_contents.close()
            fileout.write(data)
    
        fileout.close()
# Make final CSV 
# MAKE CSV FILE WITH PANDAS
        import pandas as pd
        df=pd.read_csv("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/%s.txt"%(filename), sep=' ')
        df.to_csv("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/%s.csv"%(filename), sep='\t', index=False)
        for file_ in tempfiles:
            os.remove(file_)


    
# Make the Behavioral 


In [ ]:
sub_list = [] 
for x in all_func:
    if x.split('/')[-7] not in sub_list:
        sub_list.append(x.split('/')[-7])
subj_dict = {}
for x in sub_list: 
    subj_dict[x] = [] 
    
# -add corresponding milkshake label    
ct=0
for d in subj_dict:
    task_dirs = glob.glob(os.path.join('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives', d, "ses-*/func/Analysis/feat1/*"))
    for task in task_dirs:
        dir_name = task.split("/")[-1]
        sess = task.split("/")[-5]
        mlk = dir_name.split(".")[0].split("e")[1]
        id_ = sess+"_"+mlk
        subj_dict[d].append(id_)
        ct=ct+1
        
temp_out = "/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/temp"
for sub in subj_dict:
    file = open(temp_out+"/tempfile_%s.txt"%sub, 'a')
    for label in sorted(subj_dict[sub]):
        fileX = '/projects/niblab/data/eric_data/ev_files/milkshake/mk%s_attr.txt'%label.split("_")[1]
        fileX_contents = open(fileX, 'r')
        data = fileX_contents.read()
        fileX_contents.close()
        file.write(data)
    file.close()
    
tempfiles=sorted(glob.glob("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/temp/tempfile_*.txt"))

# fill in 20% of test files 
test_slice_count = round(len(tempfiles) *.2)
test_slice = sub_list[:test_slice_count]
for sub in test_slice:
    df=pd.read_csv("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/temp/tempfile_%s.txt"%sub, sep=' ',header=None)
    df[1].replace(0,1, inplace=True)
    print(df.head())
    df.to_csv("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/temp/tempfile_%s.txt"%sub, sep=' ', index=False, header=None)    

# Make initial text file    
fileout = open("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/all_waves_b1.txt", "a")
fileout.write("Label sess\n")
       
for f in tempfiles:
    print("adding file %s"%f)
    f_contents = open(f, "r")
    data=f_contents.read()
    f_contents.close()
    fileout.write(data)
    
fileout.close()
# Make final CSV 
# MAKE CSV FILE WITH PANDAS
import pandas as pd
df=pd.read_csv("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/all_waves_b1.txt", sep=' ')
df.to_csv("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/all_waves_b1.csv", sep='\t', index=False)



#### SVM decoding took 13 hours for these 9 subjects

In [ ]:
FILENAME="all_batch1.nii.gz"

DERIV_DIR = '/projects/niblab/bids_projects/Experiments/ChocoData/derivatives'
outpath = "/projects/niblab/bids_projects/Experiments/ChocoData/images"
all_func = sorted(glob.glob(os.path.join(DERIV_DIR, "sub-*", "ses-*", "func/Analysis/feat1/task*/filtered_func_data.nii.gz")))

sub_list = [] 
for x in all_func:
    if x.split('/')[-7] not in sub_list:
        sub_list.append(x.split('/')[-7])
subj_dict = {}
for x in sub_list: 
    subj_dict[x] = []



In [ ]:
FILENAME="all_batch0.nii.gz"

DERIV_DIR = '/projects/niblab/bids_projects/Experiments/ChocoData/derivatives'
outpath = "/projects/niblab/bids_projects/Experiments/ChocoData/images"
all_func = sorted(glob.glob(os.path.join(DERIV_DIR, "sub-*", "ses-*", "func/Analysis/feat1/task*/filtered_func_data.nii.gz")))

all_func = sorted(all_func[:30])

#load in all the files from the glob above, then convert them from nifti1 to nifti2\n",
ni2_funcs = (nib.Nifti2Image.from_image(nib.load(func)) for func in all_func)
#concat, this is with nibabel, but should work with nilearn too
ni2_concat = nib.concat_images(ni2_funcs, check_affines=False, axis=3)
#set the output file name
outfile=os.path.join(outpath,FILENAME)
#write the file
ni2_concat.to_filename(outfile)

In [ ]:
sub_list = [] 
for x in all_func:
    if x.split('/')[-7] not in sub_list:
        sub_list.append(x.split('/')[-7])
subj_dict = {}
for x in sub_list: 
    subj_dict[x] = [] 
    
# -add corresponding milkshake label    
ct=0
for d in subj_dict:
    task_dirs = glob.glob(os.path.join('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives', d, "ses-*/func/Analysis/feat1/*"))
    for task in task_dirs:
        dir_name = task.split("/")[-1]
        sess = task.split("/")[-5]
        mlk = dir_name.split(".")[0].split("e")[1]
        id_ = sess+"_"+mlk
        subj_dict[d].append(id_)
        ct=ct+1
        
temp_out = "/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/temp"
for sub in subj_dict:
    file = open(temp_out+"/tempfile_%s.txt"%sub, 'a')
    for label in sorted(subj_dict[sub]):
        fileX = '/projects/niblab/data/eric_data/ev_files/milkshake/mk%s_attr.txt'%label.split("_")[1]
        fileX_contents = open(fileX, 'r')
        data = fileX_contents.read()
        fileX_contents.close()
        file.write(data)
    file.close()
    
tempfiles=sorted(glob.glob("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/temp/tempfile_*.txt"))

# fill in 20% of test files 
test_slice_count = round(len(tempfiles) *.2)
test_slice = sub_list[:test_slice_count]
for sub in test_slice:
    df=pd.read_csv("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/temp/tempfile_%s.txt"%sub, sep=' ',header=None)
    df[1].replace(0,1, inplace=True)
    print(df.head())
    df.to_csv("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/temp/tempfile_%s.txt"%sub, sep=' ', index=False, header=None)    

# Make initial text file    
fileout = open("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/all_batch0.txt", "a")
fileout.write("Label sess\n")
       
for f in tempfiles:
    print("adding file %s"%f)
    f_contents = open(f, "r")
    data=f_contents.read()
    f_contents.close()
    fileout.write(data)
    
fileout.close()
# Make final CSV 
# MAKE CSV FILE WITH PANDAS
import pandas as pd
df=pd.read_csv("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/all_batch0.txt", sep=' ')
df.to_csv("/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/all_batch0.csv", sep='\t', index=False)

